## 3. Data Preparation

This script merges all the currency files and also merge the daily level crypto prices and market prices (dow jones indicator).

Furthermore, custom variables are created such as polarity, Bullish Ratio and Negative Share.

**Output is the daily level summarised data required for modeling exercise**

In [35]:
import pandas as pd
import numpy as np
import string
# !pip install --user -U nltk # if you don't have it
import nltk 
#nltk.download() # if you don't have it
from nltk.corpus import stopwords
import nltk.tokenize 
from nltk.stem import WordNetLemmatizer

#word count
from collections import Counter

#word clouds
import heapq
#from wordcloud import WordCloud, STOPWORDS
from PIL import Image
import collections
import matplotlib.cm as cm
from matplotlib import rcParams
from tqdm import tqdm

#sentiment analysis
from textblob import TextBlob

#text preprocessing
from nltk.corpus import stopwords
import nltk
from nltk.stem import WordNetLemmatizer 
nltk.download('stopwords')
nltk.download('wordnet')
from scipy import stats

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\saggarwal\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\saggarwal\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


### Cryptocurrency Prices

Daily level crypto prices are downloaded from internet across different exchanges and average prices are calculated to get final bitcoin prices 

In [2]:
Bitfinex_BTCUSD_1h = pd.read_csv('Bitfinex_BTCUSD_1h.csv', skiprows=1)
Bitfinex_BTCUSD_1h.head()

,unix,date,symbol,open,high,low,close,Volume USD,Volume BTC
0,1629504000000,2021-08-21 00:00:00,BTC/USD,49333.0,49374.0,49030.000000,49043.000000,7.577243e+06,154.502024
1,1629500400000,2021-08-20 23:00:00,BTC/USD,49223.0,49373.0,49049.996934,49330.497131,9.385369e+06,190.254907
2,1629496800000,2021-08-20 22:00:00,BTC/USD,49336.0,49375.0,48977.000000,49223.000000,1.347543e+07,273.762974
3,1629493200000,2021-08-20 21:00:00,BTC/USD,48660.0,49400.0,48660.000000,49337.000000,1.575885e+07,319.412366
4,1629489600000,2021-08-20 20:00:00,BTC/USD,48707.0,48844.0,48644.597808,48664.800914,5.058329e+06,103.942260


In [21]:
Cexio_BTCUSD_d = pd.read_csv('Cexio_BTCUSD_d.csv', skiprows=1)
Bitstamp_BTCUSD_d = pd.read_csv('Bitstamp_BTCUSD_d.csv', skiprows=1)
#Itbit_BTCUSD_1h = pd.read_csv('Itbit_BTCUSD_1h.csv', skiprows=1)
Bittrex_BTCUSD_d = pd.read_csv('Bittrex_BTCUSD_d.csv', skiprows=1)
gemini_BTCUSD_day = pd.read_csv('gemini_BTCUSD_day.csv', skiprows=1)
gemini_BTCUSD_day.head()


,Unix Timestamp,Date,Symbol,Open,High,Low,Close,Volume
0,1629432000000,2021-08-20 04:00:00,BTCUSD,47042.28,49400.00,46714.57,49191.11,1152.981240
1,1629345600000,2021-08-19 04:00:00,BTCUSD,44074.77,47424.13,43902.53,47042.28,1566.004833
2,1629259200000,2021-08-18 04:00:00,BTCUSD,45078.79,46031.00,44055.00,44074.77,1324.725961
3,1629172800000,2021-08-17 04:00:00,BTCUSD,46435.44,47162.94,44211.13,45078.79,2149.379858
4,1629086400000,2021-08-16 04:00:00,BTCUSD,47400.89,47744.50,45250.51,46435.44,1687.082115


In [22]:
#Bitfinex_BTCUSD_1h.rename(columns={'close':'Close_Bitfinex', 'Volume BTC':'Volume_Bitfinex'},inplace=True)
#Cexio_BTCUSD_d.rename(columns={'Close':'Close_Cexio', 'Volume BTC':'Volume_Cexio'},inplace=True)
Bitstamp_BTCUSD_d.rename(columns={'close':'Close_Bitstamp', 'Volume BTC':'Volume_Bitstamp', 'open':'Open_Bitstamp',
                                 'high':'High_Bitstamp','low':'Low_Bitstamp'},inplace=True)
#Itbit_BTCUSD_1h.rename(columns={'Close':'Close_Itbit', 'Volume BTC':'Volume_Itbit'},inplace=True)
#Bittrex_BTCUSD_d.rename(columns={'Close':'Close_Bittrex', 'Volume BTC':'Volume_Bittrex'},inplace=True)
gemini_BTCUSD_day.rename(columns={'Close':'Close_gemini', 'Volume':'Volume_gemini','Open':'Open_gemini',
                                 'High':'High_gemini','Low':'Low_gemini'},inplace=True)

In [23]:
gemini_BTCUSD_day.head()

,Unix Timestamp,Date,Symbol,Open_gemini,High_gemini,Low_gemini,Close_gemini,Volume_gemini
0,1629432000000,2021-08-20 04:00:00,BTCUSD,47042.28,49400.00,46714.57,49191.11,1152.981240
1,1629345600000,2021-08-19 04:00:00,BTCUSD,44074.77,47424.13,43902.53,47042.28,1566.004833
2,1629259200000,2021-08-18 04:00:00,BTCUSD,45078.79,46031.00,44055.00,44074.77,1324.725961
3,1629172800000,2021-08-17 04:00:00,BTCUSD,46435.44,47162.94,44211.13,45078.79,2149.379858
4,1629086400000,2021-08-16 04:00:00,BTCUSD,47400.89,47744.50,45250.51,46435.44,1687.082115


In [28]:
#BTC_prices  = gemini_BTCUSD_day[['Date','Close_gemini','Volume_gemini']].merge(Bitfinex_BTCUSD_1h[['date','Close_Bitfinex','Volume_Bitfinex']], left_on ='Date',right_on='date', how='left')
gemini_BTCUSD_day['Date'] = pd.to_datetime(gemini_BTCUSD_day['Date']).dt.date
Bitstamp_BTCUSD_d['Date'] = pd.to_datetime(Bitstamp_BTCUSD_d['date']).dt.date
#Cexio_BTCUSD_d['Date'] = pd.to_datetime(Cexio_BTCUSD_d['Date']).dt.date
#Bittrex_BTCUSD_d['Date'] = pd.to_datetime(Bittrex_BTCUSD_d['Date']).dt.date
#BTC_prices  = gemini_BTCUSD_day[['Date','Close_gemini','Volume_gemini']].merge(Cexio_BTCUSD_d[['Date','Close_Cexio','Volume_Cexio']], on='Date', how='left')
BTC_prices  = gemini_BTCUSD_day[['Date','Open_gemini','High_gemini','Low_gemini','Close_gemini','Volume_gemini']].merge(Bitstamp_BTCUSD_d[['Date','Open_Bitstamp','High_Bitstamp','Low_Bitstamp','Close_Bitstamp','Volume_Bitstamp']], on='Date', how='left')
#BTC_prices  = BTC_prices.merge(Itbit_BTCUSD_1h[['Date','Close_Itbit','Volume_Itbit']], on='Date', how='left')
#BTC_prices  = BTC_prices.merge(Bittrex_BTCUSD_d[['Date','Close_Bittrex','Volume_Bittrex']], on='Date', how='left')
#BTC_prices  = BTC_prices.merge(Itbit_BTCUSD_1h[['Date','Close_gemini','Volume_gemini']], on='Date', how='left')
BTC_prices.head()

,Date,Open_gemini,High_gemini,Low_gemini,Close_gemini,Volume_gemini,Open_Bitstamp,High_Bitstamp,Low_Bitstamp,Close_Bitstamp,Volume_Bitstamp
0,2021-08-20,47042.28,49400.00,46714.57,49191.11,1152.981240,46759.29,49436.10,46645.77,49356.00,2815.551595
1,2021-08-19,44074.77,47424.13,43902.53,47042.28,1566.004833,44734.16,47114.99,43935.54,46764.30,2776.237685
2,2021-08-18,45078.79,46031.00,44055.00,44074.77,1324.725961,44631.47,46041.62,44218.73,44721.13,2247.934158
3,2021-08-17,46435.44,47162.94,44211.13,45078.79,2149.379858,45930.48,47183.37,44417.51,44694.35,3286.273265
4,2021-08-16,47400.89,47744.50,45250.51,46435.44,1687.082115,47024.96,48075.22,45654.14,45930.51,2410.150995


In [29]:
BTC_prices['Volume_BTC_gemini'] = BTC_prices['Volume_gemini']*BTC_prices['Close_gemini']
#BTC_prices['Volume_BTC_Bittrex'] = BTC_prices['Volume_Bittrex']*BTC_prices['Close_Bittrex']
BTC_prices['Volume_BTC_Bitstamp'] = BTC_prices['Volume_Bitstamp']*BTC_prices['Close_Bitstamp']
#BTC_prices['Volume_BTC_Cexio'] = BTC_prices['Volume_Cexio']*BTC_prices['Close_Cexio']
BTC_prices['Close_prices'] = (BTC_prices['Close_gemini']+BTC_prices['Close_Bitstamp'])/2
BTC_prices['Open_prices'] = (BTC_prices['Open_gemini']+BTC_prices['Open_Bitstamp'])/2
BTC_prices['High_prices'] = (BTC_prices['High_gemini']+BTC_prices['High_Bitstamp'])/2
BTC_prices['Low_prices'] = (BTC_prices['Low_gemini']+BTC_prices['Low_Bitstamp'])/2
BTC_prices['Volume_BTC'] = BTC_prices['Volume_BTC_gemini']+BTC_prices['Volume_BTC_Bitstamp']
BTC_prices.head()

,Date,Open_gemini,High_gemini,Low_gemini,Close_gemini,Volume_gemini,Open_Bitstamp,High_Bitstamp,Low_Bitstamp,Close_Bitstamp,Volume_Bitstamp,Volume_BTC_gemini,Volume_BTC_Bitstamp,Close_prices,Open_prices,High_prices,Low_prices,Volume_BTC
0,2021-08-20,47042.28,49400.00,46714.57,49191.11,1152.981240,46759.29,49436.10,46645.77,49356.00,2815.551595,5.671643e+07,1.389644e+08,49273.555,46900.785,49418.050,46680.170,1.956808e+08
1,2021-08-19,44074.77,47424.13,43902.53,47042.28,1566.004833,44734.16,47114.99,43935.54,46764.30,2776.237685,7.366844e+07,1.298288e+08,46903.290,44404.465,47269.560,43919.035,2.034972e+08
2,2021-08-18,45078.79,46031.00,44055.00,44074.77,1324.725961,44631.47,46041.62,44218.73,44721.13,2247.934158,5.838699e+07,1.005302e+08,44397.950,44855.130,46036.310,44136.865,1.589171e+08
3,2021-08-17,46435.44,47162.94,44211.13,45078.79,2149.379858,45930.48,47183.37,44417.51,44694.35,3286.273265,9.689144e+07,1.468778e+08,44886.570,46182.960,47173.155,44314.320,2.437693e+08
4,2021-08-16,47400.89,47744.50,45250.51,46435.44,1687.082115,47024.96,48075.22,45654.14,45930.51,2410.150995,7.834040e+07,1.106995e+08,46182.975,47212.925,47909.860,45452.325,1.890399e+08


### Merge different datasets

Custom variable creating and summary data at daily level is created

In [9]:
def data_manipulation(df):

    df['length_tweet'] = df['tweet_new2'].apply(len)
    df['Date'] = pd.to_datetime(df['created_date']).dt.date
    summarised = df.groupby(['Date'], as_index=False).agg({'neg':'mean','neu':'mean','pos':'mean','compound':'mean','retweet_count':'mean','like_count':'mean','author id':'count',
                                                                                  "unique_id": pd.Series.nunique, 'length_tweet':'mean','followers_count':'mean'})
    
    data_sum = df.groupby(['Date','sentiment'], as_index=False)['unique_id'].count()
    data_sum1 = data_sum.pivot(index='Date', columns='sentiment', values='unique_id')
    data_sum1['Total_hr_tweets'] = data_sum1['negative']+data_sum1['positive']+data_sum1['neutral']
    data_sum1 = data_sum1.reset_index()
    
    summarised = summarised.merge(data_sum1, on='Date', how='left')
 
    summarised['Bullish_Ratio'] = summarised['positive']/summarised['negative']
    summarised['Negative_share'] = summarised['negative']/summarised['Total_hr_tweets']
    summarised = summarised.rename(columns={'neg':'avg_neg_scr','neu':'avg_neu_scr','pos':'avg_pos_scr','compound':'avg_cmp_scr','retweet_count':'avg_retweet_cnt','like_count':'avg_like_cnt','author id':'tweet_vol',
                                                                                  "unique_id": 'uniq_tweet_vol', 'length_tweet':'avg_tweet_len','followers_count':'avg_followers_cnt'})
    #summarised['created_date']=pd.to_datetime(summarised['D']).dt.tz_localize(None)
    return summarised

final_data_senti1=pd.read_csv('final_model_file1.csv')
summarised_1 = data_manipulation(final_data_senti1)

C:\Users\saggarwal\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3146: DtypeWarning: Columns (16) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [10]:
summarised_1.head()

,Date,avg_neg_scr,avg_neu_scr,avg_pos_scr,avg_cmp_scr,avg_retweet_cnt,avg_like_cnt,tweet_vol,uniq_tweet_vol,avg_tweet_len,avg_followers_cnt,negative,neutral,positive,Total_hr_tweets,Bullish_Ratio,Negative_share
0,2021-01-29,0.025783,0.863776,0.110271,0.222172,717.124940,2.897826,35273,19832,117.777932,5236.702342,3975,14732,16566,35273,4.167547,0.112692
1,2021-01-30,0.024148,0.867288,0.107932,0.232728,1095.482829,2.114958,54046,28000,126.151371,3611.842208,5668,22449,25929,54046,4.574629,0.104874
2,2021-01-31,0.027110,0.883143,0.089500,0.181351,512.889540,2.242481,47447,26746,128.907476,3545.988324,5163,21281,21003,47447,4.067984,0.108816
3,2021-07-01,0.021007,0.877197,0.101582,0.220600,2096.803213,1.747746,121675,54104,122.808901,2179.789094,9192,60257,52226,121675,5.681680,0.075546
4,2021-07-02,0.016984,0.889221,0.093512,0.210509,2542.598358,1.644665,109249,45549,124.999753,2010.086582,7375,55318,46556,109249,6.312678,0.067506


In [11]:
final_data_senti1=pd.read_csv('final_data_senti_july31.csv')
summarised_2 = data_manipulation(final_data_senti1)
summarised_2.head()

,Date,avg_neg_scr,avg_neu_scr,avg_pos_scr,avg_cmp_scr,avg_retweet_cnt,avg_like_cnt,tweet_vol,uniq_tweet_vol,avg_tweet_len,avg_followers_cnt,negative,neutral,positive,Total_hr_tweets,Bullish_Ratio,Negative_share
0,2021-07-15,0.035304,0.818266,0.146299,0.207747,2243.650001,2.167564,322175,91927,101.980237,3050.646316,62286,110409,149480,322175,2.399897,0.193330
1,2021-07-16,0.022046,0.861828,0.116018,0.236538,2160.457427,2.249251,238567,89944,114.768203,3384.730784,21474,94845,122248,238567,5.692838,0.090012
2,2021-07-17,0.019231,0.871885,0.108516,0.229548,1329.250028,2.953045,187671,76578,119.665830,3266.766490,16239,89053,82379,187671,5.072911,0.086529
3,2021-07-18,0.017338,0.870968,0.111542,0.231115,1237.048825,2.074234,214725,91014,115.418931,3092.653301,15561,105799,93365,214725,5.999936,0.072469
4,2021-07-19,0.018112,0.868634,0.112881,0.241119,1874.924711,2.652030,241590,98044,119.175065,3407.855354,19287,107446,114857,241590,5.955151,0.079834


In [12]:
final_data_senti1=pd.read_csv('final_data_senti_aug15.csv')
summarised_3 = data_manipulation(final_data_senti1)
summarised_3.head()

,Date,avg_neg_scr,avg_neu_scr,avg_pos_scr,avg_cmp_scr,avg_retweet_cnt,avg_like_cnt,tweet_vol,uniq_tweet_vol,avg_tweet_len,avg_followers_cnt,negative,neutral,positive,Total_hr_tweets,Bullish_Ratio,Negative_share
0,2021-08-01,0.018967,0.865553,0.115214,0.253800,2415.091440,2.276525,273632,104006,118.413373,3348.865995,19244,117986,136402,273632,7.088027,0.070328
1,2021-08-02,0.017276,0.870965,0.111387,0.239241,1077.593054,2.151700,263079,115789,119.697403,3605.798064,19525,121537,122017,263079,6.249270,0.074217
2,2021-08-03,0.020620,0.869579,0.109591,0.224879,1663.179273,2.798036,253664,101746,119.605041,4159.025333,22710,116756,114198,253664,5.028534,0.089528
3,2021-08-04,0.015083,0.849022,0.135521,0.306233,3287.812946,2.630993,312333,102652,118.731293,2787.742499,20107,119747,172479,312333,8.578057,0.064377
4,2021-08-05,0.015767,0.845788,0.138001,0.313645,4606.569417,1.987932,324983,109644,119.001680,2892.571267,21042,120443,183498,324983,8.720559,0.064748


In [30]:
summarised_all=summarised_1.append(summarised_2)
summarised_all=summarised_all.append(summarised_3)
summarised_all.shape

(49, 17)

In [31]:
#summarised_all['created_date']=pd.to_datetime(summarised_all['created_date'])
summarised_all.head()

,Date,avg_neg_scr,avg_neu_scr,avg_pos_scr,avg_cmp_scr,avg_retweet_cnt,avg_like_cnt,tweet_vol,uniq_tweet_vol,avg_tweet_len,avg_followers_cnt,negative,neutral,positive,Total_hr_tweets,Bullish_Ratio,Negative_share
0,2021-01-29,0.025783,0.863776,0.110271,0.222172,717.124940,2.897826,35273,19832,117.777932,5236.702342,3975,14732,16566,35273,4.167547,0.112692
1,2021-01-30,0.024148,0.867288,0.107932,0.232728,1095.482829,2.114958,54046,28000,126.151371,3611.842208,5668,22449,25929,54046,4.574629,0.104874
2,2021-01-31,0.027110,0.883143,0.089500,0.181351,512.889540,2.242481,47447,26746,128.907476,3545.988324,5163,21281,21003,47447,4.067984,0.108816
3,2021-07-01,0.021007,0.877197,0.101582,0.220600,2096.803213,1.747746,121675,54104,122.808901,2179.789094,9192,60257,52226,121675,5.681680,0.075546
4,2021-07-02,0.016984,0.889221,0.093512,0.210509,2542.598358,1.644665,109249,45549,124.999753,2010.086582,7375,55318,46556,109249,6.312678,0.067506


In [32]:
summarised_all = summarised_all.merge(BTC_prices[['Date','Open_prices', 'High_prices','Low_prices','Close_prices','Volume_BTC']], left_on='Date', right_on='Date',how='left')
summarised_all.head()

,Date,avg_neg_scr,avg_neu_scr,avg_pos_scr,avg_cmp_scr,avg_retweet_cnt,avg_like_cnt,tweet_vol,uniq_tweet_vol,avg_tweet_len,...,neutral,positive,Total_hr_tweets,Bullish_Ratio,Negative_share,Open_prices,High_prices,Low_prices,Close_prices,Volume_BTC
0,2021-01-29,0.025783,0.863776,0.110271,0.222172,717.124940,2.897826,35273,19832,117.777932,...,14732,16566,35273,4.167547,0.112692,33676.335,38642.855,31979.135,33769.385,1.334235e+09
1,2021-01-30,0.024148,0.867288,0.107932,0.232728,1095.482829,2.114958,54046,28000,126.151371,...,22449,25929,54046,4.574629,0.104874,33777.420,34868.520,33006.585,34136.395,2.967333e+08
2,2021-01-31,0.027110,0.883143,0.089500,0.181351,512.889540,2.242481,47447,26746,128.907476,...,21281,21003,47447,4.067984,0.108816,34124.990,34292.605,32196.850,33375.080,1.961919e+08
3,2021-07-01,0.021007,0.877197,0.101582,0.220600,2096.803213,1.747746,121675,54104,122.808901,...,60257,52226,121675,5.681680,0.075546,34595.875,34773.445,32711.785,33248.825,1.784648e+08
4,2021-07-02,0.016984,0.889221,0.093512,0.210509,2542.598358,1.644665,109249,45549,124.999753,...,55318,46556,109249,6.312678,0.067506,33235.275,33953.980,32700.025,33676.020,1.148738e+08


In [33]:
summarised_all['Date'] = pd.to_datetime(summarised_all['Date'])
summarised_all

,Date,avg_neg_scr,avg_neu_scr,avg_pos_scr,avg_cmp_scr,avg_retweet_cnt,avg_like_cnt,tweet_vol,uniq_tweet_vol,avg_tweet_len,...,neutral,positive,Total_hr_tweets,Bullish_Ratio,Negative_share,Open_prices,High_prices,Low_prices,Close_prices,Volume_BTC
0,2021-01-29,0.025783,0.863776,0.110271,0.222172,717.124940,2.897826,35273,19832,117.777932,...,14732,16566,35273,4.167547,0.112692,33676.335,38642.855,31979.135,33769.385,1.334235e+09
1,2021-01-30,0.024148,0.867288,0.107932,0.232728,1095.482829,2.114958,54046,28000,126.151371,...,22449,25929,54046,4.574629,0.104874,33777.420,34868.520,33006.585,34136.395,2.967333e+08
2,2021-01-31,0.027110,0.883143,0.089500,0.181351,512.889540,2.242481,47447,26746,128.907476,...,21281,21003,47447,4.067984,0.108816,34124.990,34292.605,32196.850,33375.080,1.961919e+08
3,2021-07-01,0.021007,0.877197,0.101582,0.220600,2096.803213,1.747746,121675,54104,122.808901,...,60257,52226,121675,5.681680,0.075546,34595.875,34773.445,32711.785,33248.825,1.784648e+08
4,2021-07-02,0.016984,0.889221,0.093512,0.210509,2542.598358,1.644665,109249,45549,124.999753,...,55318,46556,109249,6.312678,0.067506,33235.275,33953.980,32700.025,33676.020,1.148738e+08
5,2021-07-03,0.015089,0.886210,0.098290,0.215499,5788.559491,1.976620,126263,44563,122.904485,...,62637,55244,126263,6.590790,0.066385,33675.470,34951.140,33399.860,34679.150,6.563334e+07
6,2021-07-04,0.014355,0.873028,0.112238,0.257782,3349.956329,1.801355,108334,40391,122.499575,...,46738,55071,108334,8.440000,0.060230,34671.040,35972.855,34307.790,34825.025,7.937801e+07
7,2021-07-05,0.016379,0.870216,0.112665,0.259466,2227.247343,1.701465,120897,50291,124.201949,...,51704,60639,120897,7.088964,0.070754,34807.920,34909.210,33121.500,33823.215,1.209090e+08
8,2021-07-06,0.018260,0.865466,0.115642,0.270794,2098.188073,1.974374,123314,48703,122.262249,...,46061,66354,123314,6.088081,0.088384,33836.490,35107.405,33520.490,34354.720,1.520159e+08
9,2021-07-07,0.014177,0.893831,0.091727,0.208712,2605.157232,1.723612,110175,46231,123.508963,...,56419,46998,110175,6.954424,0.061339,34348.650,35078.650,33447.320,33609.735,1.017113e+08


In [38]:
summarised_all['polarity'] = stats.gmean(summarised_all.iloc[:, [1,3]], axis=1)

In [40]:
summarised_all.head()
summarised_all.columns

Index(['Date', 'avg_neg_scr', 'avg_neu_scr', 'avg_pos_scr', 'avg_cmp_scr',
       'avg_retweet_cnt', 'avg_like_cnt', 'tweet_vol', 'uniq_tweet_vol',
       'avg_tweet_len', 'avg_followers_cnt', 'negative', 'neutral', 'positive',
       'Total_hr_tweets', 'Bullish_Ratio', 'Negative_share', 'Open_prices',
       'High_prices', 'Low_prices', 'Close_prices', 'Volume_BTC', 'polarity'],
      dtype='object')

In [41]:
summarised_all = summarised_all[summarised_all['Date'] >= '2021-07-01'][['Date', 'avg_neg_scr', 'avg_neu_scr', 'avg_pos_scr', 'avg_cmp_scr',
       'avg_retweet_cnt', 'avg_like_cnt', 'tweet_vol', 'uniq_tweet_vol',
       'avg_tweet_len', 'avg_followers_cnt', 'negative', 'neutral', 'positive',
       'Bullish_Ratio', 'Negative_share', 'Open_prices',
       'High_prices', 'Low_prices', 'Close_prices', 'Volume_BTC', 'polarity']]
summarised_all.to_csv('model_file_final_btc.csv')

In [44]:
summarised_all['pct_change'] = summarised_all.Close_prices.pct_change()

In [45]:
def direction(df):

    if (df['pct_change'] <0):
        return 0
    elif (df['pct_change'] >=0):
        return 1

summarised_all['direction']= summarised_all.apply(direction, axis = 1)

In [20]:
summarised_all.head()

,Date,avg_neg_scr,avg_neu_scr,avg_pos_scr,avg_cmp_scr,avg_retweet_cnt,avg_like_cnt,tweet_vol,uniq_tweet_vol,avg_tweet_len,...,negative,neutral,positive,Total_hr_tweets,Bullish_Ratio,Negative_share,Close_prices,Volume_BTC,pct_change,direction
0,2021-01-29,0.025783,0.863776,0.110271,0.222172,717.124940,2.897826,35273,19832,117.777932,...,3975,14732,16566,35273,4.167547,0.112692,33769.385,1.334235e+09,NaN,NaN
1,2021-01-30,0.024148,0.867288,0.107932,0.232728,1095.482829,2.114958,54046,28000,126.151371,...,5668,22449,25929,54046,4.574629,0.104874,34136.395,2.967333e+08,0.010868,1.0
2,2021-01-31,0.027110,0.883143,0.089500,0.181351,512.889540,2.242481,47447,26746,128.907476,...,5163,21281,21003,47447,4.067984,0.108816,33375.080,1.961919e+08,-0.022302,0.0
3,2021-07-01,0.021007,0.877197,0.101582,0.220600,2096.803213,1.747746,121675,54104,122.808901,...,9192,60257,52226,121675,5.681680,0.075546,33248.825,1.784648e+08,-0.003783,0.0
4,2021-07-02,0.016984,0.889221,0.093512,0.210509,2542.598358,1.644665,109249,45549,124.999753,...,7375,55318,46556,109249,6.312678,0.067506,33676.020,1.148738e+08,0.012848,1.0


### dow jones prices

Daily level market prices are merged with the summarised tweet dataset

In [50]:
dow=pd.read_csv('DOW_Barchart_Interactive_Chart_Daily_08_23_2021.csv', skiprows=[0,615])
dow.tail()

,Date Time,Open,High,Low,Close,Change,Volume,Open Interest
608,2021-08-17,62.69,62.875,61.14,62.31,-1.01,3974400,NaN
609,2021-08-18,61.91,62.830,61.71,61.88,-0.43,4573900,NaN
610,2021-08-19,61.04,61.390,60.15,60.55,-1.33,3981500,NaN
611,2021-08-20,60.41,61.110,60.32,60.89,0.34,3203600,NaN
612,2021-08-23,61.50,62.120,61.46,62.25,1.36,1150601,NaN


In [59]:
dow['Date']=pd.to_datetime(dow['Date Time'])
dow['index_change']=dow['Close'].pct_change()
dow.head()

,Date Time,Open,High,Low,Close,Change,Volume,Open Interest,Date,index_change
0,2019-03-20,52.75,53.50,49.50,49.80,0.00,2350800,NaN,2019-03-20,NaN
1,2019-03-21,49.99,50.00,48.20,48.98,-0.82,1764600,NaN,2019-03-21,-0.016466
2,2019-03-22,48.80,49.95,48.16,48.60,-0.38,844600,NaN,2019-03-22,-0.007758
3,2019-03-25,48.60,49.40,48.00,49.15,0.55,440800,NaN,2019-03-25,0.011317
4,2019-03-26,49.00,49.75,48.18,48.85,-0.30,504700,NaN,2019-03-26,-0.006104


In [61]:
#summarised_all=summarised_all.merge(dow[['Date','Close']], on='Date', how='left')
summarised_all['Close'] = summarised_all['Close'].fillna(method='ffill')
summarised_all.head()

,Date,avg_neg_scr,avg_neu_scr,avg_pos_scr,avg_cmp_scr,avg_retweet_cnt,avg_like_cnt,tweet_vol,uniq_tweet_vol,avg_tweet_len,...,High_prices,Low_prices,Close_prices,Volume_BTC,polarity,pct_change,direction,Close_x,Close_y,Close
0,2021-07-01,0.021007,0.877197,0.101582,0.220600,2096.803213,1.747746,121675,54104,122.808901,...,34773.445,32711.785,33248.825,1.784648e+08,0.046195,NaN,NaN,63.50,63.50,63.50
1,2021-07-02,0.016984,0.889221,0.093512,0.210509,2542.598358,1.644665,109249,45549,124.999753,...,33953.980,32700.025,33676.020,1.148738e+08,0.039852,0.012848,1.0,63.51,63.51,63.51
2,2021-07-03,0.015089,0.886210,0.098290,0.215499,5788.559491,1.976620,126263,44563,122.904485,...,34951.140,33399.860,34679.150,6.563334e+07,0.038510,0.029788,1.0,0.00,NaN,63.51
3,2021-07-04,0.014355,0.873028,0.112238,0.257782,3349.956329,1.801355,108334,40391,122.499575,...,35972.855,34307.790,34825.025,7.937801e+07,0.040139,0.004206,1.0,0.00,NaN,63.51
4,2021-07-05,0.016379,0.870216,0.112665,0.259466,2227.247343,1.701465,120897,50291,124.201949,...,34909.210,33121.500,33823.215,1.209090e+08,0.042958,-0.028767,0.0,0.00,NaN,63.51


In [66]:
#summarised_all = summarised_all.drop(columns =['Close_x', 'Close_y'])

In [64]:
#import libraries
import pandas as pd
import numpy as np
import seaborn as sns
import statsmodels.api as sm
import matplotlib.pyplot as plt
#features selection and class imbalance
from sklearn.feature_selection import RFE, VarianceThreshold
from sklearn.kernel_approximation import RBFSampler
from sklearn.preprocessing import StandardScaler
from sklearn.kernel_approximation import PolynomialCountSketch
from sklearn.pipeline import Pipeline, make_pipeline
from imblearn.over_sampling import SMOTE
#machine learning models
from sklearn.linear_model import LogisticRegression
from xgboost import XGBClassifier
from sklearn.linear_model import SGDClassifier
from sklearn.svm import LinearSVC
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import RandomForestClassifier
from catboost import CatBoostClassifier
#from lightgbm import LGBMClassifier
from sklearn.ensemble import StackingClassifier
#model performance evaluation
from sklearn.model_selection import train_test_split, GridSearchCV, RepeatedStratifiedKFold, cross_val_score
from sklearn.metrics import ConfusionMatrixDisplay, roc_auc_score, roc_curve, confusion_matrix
from sklearn.metrics import classification_report, accuracy_score

In [27]:
summarised_all = summarised_all.dropna()

In [67]:
summarised_all.head()

,Date,avg_neg_scr,avg_neu_scr,avg_pos_scr,avg_cmp_scr,avg_retweet_cnt,avg_like_cnt,tweet_vol,uniq_tweet_vol,avg_tweet_len,...,Negative_share,Open_prices,High_prices,Low_prices,Close_prices,Volume_BTC,polarity,pct_change,direction,Close
0,2021-07-01,0.021007,0.877197,0.101582,0.220600,2096.803213,1.747746,121675,54104,122.808901,...,0.075546,34595.875,34773.445,32711.785,33248.825,1.784648e+08,0.046195,NaN,NaN,63.50
1,2021-07-02,0.016984,0.889221,0.093512,0.210509,2542.598358,1.644665,109249,45549,124.999753,...,0.067506,33235.275,33953.980,32700.025,33676.020,1.148738e+08,0.039852,0.012848,1.0,63.51
2,2021-07-03,0.015089,0.886210,0.098290,0.215499,5788.559491,1.976620,126263,44563,122.904485,...,0.066385,33675.470,34951.140,33399.860,34679.150,6.563334e+07,0.038510,0.029788,1.0,63.51
3,2021-07-04,0.014355,0.873028,0.112238,0.257782,3349.956329,1.801355,108334,40391,122.499575,...,0.060230,34671.040,35972.855,34307.790,34825.025,7.937801e+07,0.040139,0.004206,1.0,63.51
4,2021-07-05,0.016379,0.870216,0.112665,0.259466,2227.247343,1.701465,120897,50291,124.201949,...,0.070754,34807.920,34909.210,33121.500,33823.215,1.209090e+08,0.042958,-0.028767,0.0,63.51


In [236]:
summarised_all.to_csv('input_model_file.csv')

In [237]:
summarised_all.shape

(46, 25)